<a href="https://colab.research.google.com/github/lddm/forests-monitoring/blob/main/GEE_LS8_explorer.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [40]:
# Installs geemap package
import subprocess

try:
        import geemap
except ImportError:
        print('geemap package not installed. Installing ...')
        subprocess.check_call(["python", '-m', 'pip', 'install', 'geemap'])


geemap package not installed. Installing ...


In [1]:
import ee

# Trigger the authentication flow.
ee.Authenticate()

# Initialize the library.
ee.Initialize()

To authorize access needed by Earth Engine, open the following URL in a web browser and follow the instructions. If the web browser does not start automatically, please manually browse the URL below.

    https://accounts.google.com/o/oauth2/auth?client_id=517222506229-vsmmajv00ul0bs7p89v5m89qs8eb9359.apps.googleusercontent.com&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fearthengine+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdevstorage.full_control&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&response_type=code&code_challenge=q5h_ix4qEuifUO5tQW21RNx363jICcQcMnwhOzIOga0&code_challenge_method=S256

The authorization workflow will generate a code, which you should paste in the box below. 
Enter verification code: 4/1AY0e-g54ZyFBdxoBGzhA8y0-M2oXoe7XECwinIUHlLvf4PdHG9aecADIShY

Successfully saved authorization token.


In [107]:

ls8 = ee.ImageCollection("LANDSAT/LC08/C01/T1_SR")
# Initial date of interest (inclusive).
i_date = '2015-01-01'

# Final date of interest (exclusive).
f_date = '2015-01-31'

rgb_bands = ['B4', 'B3', 'B2']

ls8 = ls8.select(rgb_bands).filterDate(i_date, f_date)

In [108]:
roi_center_lon = -55.596655
roi_center_lat = -31.388125
roi = ee.Geometry.Rectangle(roi_center_lon - 0.001,
                      roi_center_lat - 0.001,
                      roi_center_lon + 0.001,
                      roi_center_lat + 0.001)

print('Before ROI filtering:', ls8.size().getInfo())
ls8 = ls8.filterBounds(roi)
print('After ROI filtering:', ls8.size().getInfo())

Before ROI filtering: 8703
After ROI filtering: 4


In [109]:
# This is OK for small collections
# print(ls8.size())
if ls8.size().getInfo() == 0:
  raise ValueError('No rasters for the selected ROI and date range were found')
ls8_collection_list = ls8.toList(ls8.size())
collection_size = ls8_collection_list.size().getInfo()

map = geemap.Map()
ls8_vis = {'min': 0, 'max': 3000, 'opacity': 1.0, 'gamma': 1.4}
map.set_center(-54.8712, -31.7001, 8)

for i in range(collection_size):
  raster_img = ee.Image(ls8_collection_list.get(i)).clip(region)
  map.addLayer(raster_img, ls8_vis, 'LS8-' + str(i))
map.addLayerControl()
map

In [50]:
import geemap
Map = geemap.Map()

ls8_img = ls8.first()
ls8_vis = {'min': 0, 'max': 3000, 'opacity': 1.0, 'gamma': 1.4}
Map.addLayer(ls8_img, ls8_vis, 'LS8')
Map.set_center(-54.8712, -31.7001, 8)
Map.addLayerControl()
Map